# Models visualised

In [ ]:
import datetime as dt
import logging
import pathlib

import numpy as np
import xarray as xr
import pandas as pd
# import matplotlib as mpl
# import matplotlib.pyplot as plt
# from scipy.interpolate import interp1d
# import cartopy.crs as ccrs
# import cartopy.feature as cfeature
import holoviews as hv
import hvplot.xarray
import panel as pn

import eoxmagmod
from chaosmagpy.plot_utils import nio_colormap

logger = logging.getLogger("model-explorer")

## Clear cache if necessary

Should be in `./cache`. The `@pn.cache(to_disk=True)` decorators further down populate this cache.

In [ ]:
# pn.state.clear_caches()

## Magnetic model evaluation tools

https://esa-vires.github.io/MagneticModel/index.html

In [ ]:
try:
    current_folder = pathlib.Path(__file__).parent
except NameError:
    current_folder = globals()['_dh'][0]

In [ ]:
mco_sha_2c_file = current_folder / pathlib.Path("./data/SW_OPER_MCO_SHA_2C_20131125T000000_20220601T000000_0801/SW_OPER_MCO_SHA_2C_20131125T000000_20220601T000000_0801.shc")
mio_sha_2c_file = current_folder / pathlib.Path("./data/SW_OPER_MIO_SHA_2C_00000000T000000_99999999T999999_0801/SW_OPER_MIO_SHA_2C_00000000T000000_99999999T999999_0801.txt")
mma_sha_2c_file = current_folder / pathlib.Path("./data/SW_OPER_MMA_SHA_2C_20131125T000000_20181231T235959_0501/SW_OPER_MMA_SHA_2C_20131125T000000_20181231T235959_0501.cdf")
mli_sha_2c_file = current_folder / pathlib.Path("./data/SW_OPER_MLI_SHA_2C_00000000T000000_99999999T999999_0801/SW_OPER_MLI_SHA_2C_00000000T000000_99999999T999999_0801.shc")

In [ ]:
mco = eoxmagmod.load_model_shc(str(mco_sha_2c_file))
mio_e = eoxmagmod.load_model_swarm_mio_external(str(mio_sha_2c_file))
mio_i = eoxmagmod.load_model_swarm_mio_internal(str(mio_sha_2c_file))
mma_e = eoxmagmod.load_model_swarm_mma_2c_external(str(mma_sha_2c_file))
mma_i = eoxmagmod.load_model_swarm_mma_2c_internal(str(mma_sha_2c_file))
mli = eoxmagmod.load_model_shc(str(mli_sha_2c_file))

In [ ]:
hv.extension("bokeh")

In [ ]:
def generate_latlon_grid(resolution=2, min_lat=-90, max_lat=90, height=0):
    "Generate a grid of positions over the Earth at a given degree resolution at a given height (km) relative to reference radius"
    lat, lon = np.meshgrid(
        np.arange(min_lat, max_lat, resolution),
        np.arange(-180, 180, resolution)
    )
    REFRAD_KM = 6371.200
    radius = np.ones_like(lat)*(height + REFRAD_KM)
    return lat, lon, radius


def datetime_to_mjd2000(t: dt.datetime) -> float:
    "Convert a datetime object to MJD2000."
    # Convert to datetime64 ns
    t = np.datetime64(t).astype("M8[ns]")
    # Get offset to year 2000
    t = (t - np.datetime64('2000')).astype('int64')
    # Convert from ns to days
    NS2DAYS = 1.0/(24*60*60*1e9)
    return t * NS2DAYS


def eval_model(
        lat: np.ndarray, lon: np.ndarray, radius: np.ndarray,
        times=np.array([dt.datetime(2018, 1, 1)]),
        shc_model=eoxmagmod.data.IGRF13,
        generic_model=None, f107=1.0
    ):
    """Evaluate a model on a grid across different times

    Returns:
        ndarray: B_NEC values at each point
    """
    # Convert Python datetime to MJD2000
    mjd2000 = [datetime_to_mjd2000(t) for t in times]
    # Load model
    model = generic_model if generic_model else eoxmagmod.load_model_shc(shc_model)
    # Reshape the input coordinates to use eoxmagmod
    orig_shape = lat.shape
    _lat, _lon, _radius = map(lambda x: x.flatten(), (lat, lon, radius))
    coords = np.stack((_lat, _lon, _radius), axis=1)
    # expand coords according to the time dimension
    coords = np.stack([coords for i in range(len(mjd2000))])
    timestack = np.stack([np.ones_like(_lat)*t for t in mjd2000])
    # Evaluate in North, East, Up coordinates
    # Specify coordinate systems according to:
    #   CT_GEODETIC_ABOVE_WGS84 = 0,
    #   CT_GEOCENTRIC_SPHERICAL = 1,
    #   CT_GEOCENTRIC_CARTESIAN = 2
    # https://github.com/ESA-VirES/MagneticModel/blob/staging/eoxmagmod/eoxmagmod/pymm_coord.h
    input_coordinate_system = 1
    output_coordinate_system = 1
    # Use scale to convert NEU to NEC
    b_nec = model.eval(timestack, coords, input_coordinate_system, output_coordinate_system, scale=[1, 1, -1], f107=f107)
    # Inclination (I), declination (D), intensity (F)
    # inc, dec, F = eoxmagmod.vincdecnorm(b_neu)
    # Reshape output back to original grid
    b_nec = b_nec.reshape(times.shape + orig_shape + (3, ))
    return b_nec


def eval_model_on_grid(model=None, heights=np.arange(0, 1000, 100), resolution=1, times=np.array([dt.datetime(2018, 1, 1)])):
    ds_set = []
    for height in heights:
        latG, lonG, radiusG = generate_latlon_grid(resolution=resolution, height=height)
        b_nec = eval_model(latG, lonG, radiusG, times=times, generic_model=model)
        _ds = xr.Dataset({'B_NEC': (['time', 'x', 'y', 'NEC'],  b_nec)},
                 coords={'lon': (['x', 'y'], lonG),
                         'lat': (['x', 'y'], latG),
                         'time': times})
        _ds = _ds.assign_coords({"height": height})
        ds_set += [_ds]
    ds = xr.concat(ds_set, dim="height")
    ds["height"].attrs = {"units": "km"}
    ds = ds.assign_coords({"NEC": np.array(["N", "E", "C"])})
    return ds

## Core field

In [ ]:
@pn.cache(to_disk=True)
def get_core_field():
    logger.warning("Evaluating core field...")
    return eval_model_on_grid(
        model=mco,
        heights=np.arange(-3000, 3000, 500),
        resolution=0.5,
    )

ds_core = get_core_field()
# ds_core

In [ ]:
def plot_slice_core(height=0, itime=0, NEC="C"):
    return ds_core.isel(time=0).sel(height=height, NEC=NEC).hvplot.scatter(
        x="lon", y="lat", c="B_NEC", by="NEC", subplots=True,
        rasterize=True, colorbar=True, hover=False, width=500, height=250, cmap=nio_colormap(), dynamic=False,
        # clabel="nT"
    ).opts(shared_axes=False)

dmap_core = (
    hv.DynamicMap(plot_slice_core, kdims=["height"])#, "NEC"])
    .redim.values(height=tuple(ds_core["height"].values),)# NEC=("N", "E", "C"))
).options(title="Core (MCO)", show_title=False)
# dmap_core

In [ ]:
# height_slider = pn.widgets.DiscreteSlider(options=list(ds_igrf["height"].values), name="height", value=0)
# dmap = hv.DynamicMap(plot_slice_core, kdims=["height"], streams=[hv.streams.Params(height_slider, ['height'])])
# pn.Column(height_slider, dmap)

## Lithospheric field

In [ ]:
@pn.cache(to_disk=True)
def get_crust_field():
    logger.warning("Evaluating crustal field...")
    return eval_model_on_grid(
        model=mli,
        heights=range(0, 600, 100),
        resolution=0.5
    )

ds_crust = get_crust_field()
# ds_crust

In [ ]:
def plot_slice_crust(height=0, itime=0, NEC="C"):
    _ds = ds_crust.sel(height=height, NEC=NEC).isel(time=itime)
    minmax = float(min(abs(_ds["B_NEC"].quantile(0.01)), abs(_ds["B_NEC"].quantile(0.99))))
    return _ds.hvplot.scatter(
        x="lon", y="lat", c="B_NEC", by="NEC", subplots=True,
        rasterize=True, colorbar=True, dynamic=False, hover=False,
        width=500, height=250, cmap=nio_colormap(), clim=(-minmax, minmax),
    ).opts(shared_axes=False)

In [ ]:
dmap_crust = (
    hv.DynamicMap(plot_slice_crust, kdims=["height",])
    .redim.values(height=tuple(ds_crust["height"].values),)
).options(title="Crust (MLI)", show_title=False)
# dmap_crust

## Ionospheric field

In [ ]:
@pn.cache(to_disk=True)
def get_mio_field():
    logger.warning("Evaluating ionospheric field...")
    heights = np.array([0])
    times = np.array([dt.datetime(2018, 1, 1, x) for x in range(0, 24, 2)])
    resolution = 0.5
    ds_mio_e = eval_model_on_grid(model=mio_e, heights=heights, times=times, resolution=resolution)
    ds_mio_i = eval_model_on_grid(model=mio_i, heights=heights, times=times, resolution=resolution)
    return xr.Dataset(
        {"B_NEC":
             xr.concat(
                (ds_mio_e["B_NEC"].expand_dims(dim="component"), ds_mio_i["B_NEC"].expand_dims(dim="component")),
                dim="component"
            ).assign_coords({"component": np.array(["external", "internal"])})
        }
    )

ds_mio = get_mio_field()
# ds_mio_e

In [ ]:
def plot_slice_mio(height=0, time=0, component="external", NEC="C"):
    _ds = ds_mio.sel(height=height, time=time, component=component, NEC=NEC)
    minmax = float(min(abs(_ds["B_NEC"].quantile(0.01)), abs(_ds["B_NEC"].quantile(0.99))))
    return _ds.hvplot.scatter(
        x="lon", y="lat", c="B_NEC", by="NEC", subplots=True,
        rasterize=True, colorbar=True, dynamic=False, hover=False,
        width=500, height=250, cmap=nio_colormap(), clim=(-minmax, minmax),
    ).opts(shared_axes=False)

dmap_mio = (
    hv.DynamicMap(plot_slice_mio, kdims=["time", "component"])
    .redim.values(time=tuple(ds_mio["time"].values), component=("external", "internal"))
).options(title="Ionosphere (MIO)", show_title=False)
# dmap_mio

## Magnetospheric field

In [ ]:
@pn.cache(to_disk=True)
def get_mma_field():
    logger.warning("Evaluating magnetospheric field...")
    heights = np.array([0])
    times = np.array([dt.datetime(2018, 1, 1, x) for x in range(0, 24, 2)])
    resolution = 0.5
    ds_mma_e = eval_model_on_grid(model=mma_e, heights=heights, times=times, resolution=resolution)
    ds_mma_i = eval_model_on_grid(model=mma_i, heights=heights, times=times, resolution=resolution)
    return xr.Dataset(
        {"B_NEC":
             xr.concat(
                (ds_mma_e["B_NEC"].expand_dims(dim="component"), ds_mma_i["B_NEC"].expand_dims(dim="component")),
                dim="component"
            ).assign_coords({"component": np.array(["external", "internal"])})
        }
    )

ds_mma = get_mma_field()
# ds_mma

In [ ]:
def plot_slice_mma(height=0, time=0, component="external", NEC="C"):
    _ds = ds_mma.sel(height=height, time=time, component=component, NEC=NEC)
    minmax = float(min(abs(_ds["B_NEC"].quantile(0.01)), abs(_ds["B_NEC"].quantile(0.99))))
    return _ds.hvplot.scatter(
        x="lon", y="lat", c="B_NEC", by="NEC", subplots=True,
        rasterize=True, colorbar=True, dynamic=False, hover=False,
        width=500, height=250, cmap=nio_colormap(), clim=(-minmax, minmax),
    ).opts(shared_axes=False)

dmap_mma = (
    hv.DynamicMap(plot_slice_mma, kdims=["time", "component"])
    .redim.values(time=tuple(ds_mma["time"].values), component=("external", "internal"))
).options(title="Magnetosphere (MMA)", show_title=False)
# dmap_mma

## Compile dashboard

In [ ]:
# spinner = pn.indicators.LoadingSpinner()
# pn.state.add_periodic_callback(lambda: spinner.param.set_param(value=pn.state.param.busy), period=100)

vires_core_url = "https://vires.services/?ws=data%3Aapplication/json%2Bgzip%3Bbase64%2CH4sIAMta5mMC/51Va0/iQBT9K6RfF5p5dF5%2BQ5R1E18Rd1UMaSodcTalJW1hNab/fe%2BMPEpZXSWZQOfMPefc3t7evnqFzhdmrH/pvDBZ6h20POpj7COv3fJKM9UDnehx%2BXZ07xGEeQdhWNcIHbjlI4SGNrp2RugBU7B8pdTQGy2ljrJpZNY6rINJh0IseU%2BHXKOtM6uzMPpPPSVvkmQP2tJmeRbPx2XRy9JHM5nn0TLi1RvchBeXx1fhWfd7Nzy9CvGhgxemMA%2BJhuvHKCk0SGTzMjGpLurQLBqb8gUQbD2iPJrqEmrlFA7D8%2BOeu8qjdGKV7jGDVNstZv9svuMsyfJiHDkf77cuvQrQ/ldJABburnUMUJnPtZVx/mGui24jiw63evyjFNbc8CyLdbLNB7Jg75ErS9%2BQNmW0aW2VDPlit2j9fQp2spMf/Wqxbhu%2ByPkS%2BqHvXaOuwpHEPzmxWeh8YtJJiB132OAS5/cZ5o8GUwJRfoJ3tCevb3QSrzq/XqN2K0Af1uc2PL5pmDJ4mP%2B3vNuXONyLWLnj6SxLdVrau7x/9YztDu%2BsdxEOTrohOXJ9YyZuqnzzdhq3qkZvMoDl0WVWmPWAmW020M8MSYqVLyQnmFLebnUIx0hwRnwWMBoQJaF9A0wQ44GPhSScBNzeQGzyzaztIJ9SFRCOFGJSAhNoyGcCCcE5SAbwS%2BARQaAIAGJcEgSWEG7F5rOlClIgISRMXImoVJJYGcEVZxQLLlQgqGAW4xRjKQVWQiglFLUquZk8le6V8xVlAmNEGZFMKY4thUibHkgoCg5KCpcOQRyMFKyAQP6SjWzhnrvPptia3acwo8t5/Da%2Bk6xsjm54SC%2BW5D4ZfTf/3f40enDDJ50nicXIKmi03jVCXGusoiw2qoPEcSHHLb/acBztuHxK0GJFVOokMeXy3eoms6eo9n05zKNFVtv3nqI8MfWP0vmgV9v9hDpF8Qqoqr%2BP2UcCwQcAAA%3D%3D"
vires_crust_url = "https://vires.services/?ws=data%3Aapplication/json%2Bgzip%3Bbase64%2CH4sIAMta5mMC/51Va0/iQBT9K6RfF5p5dF5%2BQ5TVBF0j7qoY0lQ64mxKS9rCagj/fe%2BMPEpZXSWZQOfMPefcuZ3eWXiFzudmpH/pvDBZ6h01POpj7COv2fBKM9F9nehR%2Bbb04BGEeQthGDcIHbnhI4QGNrqyRugRUzB8pdTAG66kTrJJZDY6rIVJi0IseU%2BH3KCdNaszN/pPNSVvnGSP2tKmeRbPRmXRydInM57l0Spi4fVvwx9Xp9fhRft7O%2Bxdh/jYwXNTmMdEw/NTlBQaJLJZmZhUF1VoGo1M%2BQoIth5RHk10CbVyCsfh5WnHPeVROrZKD5hBqs0Gs38231GWZHkxipyP91uX3hLQ7ldJABZu1zoGqMxn2so4/zDXRbuWRYtbPf5RChtueJHFOtnlA1mw98hLS9%2BStmW0ae2UDPliv2jdQwp2tpcfxl%2Bp1F3NFDlTQj80va8VVTiS%2BCcnNnOdj006DrHjDmpc4vw%2BwzyvMSUQ5Sd4JwfyukYn8frYV2vUbATow/rchae3NVMGb/L/lveHEgcHEZdueTLNUp2WdpcPC8/Y0%2BFd9M7D/lk7JCfu3JixaynfvL1Tu1wO32QAy6OrrDCb7jLdTuAwMyQpVr6QnGBKebPRIhwjwRnxWcBoQJRsNmiACWI88LGQhJOA2w3EJt822hbyKVUB4UghJiUwgYZ8JpAQnINkAL8EXhEEigAgxiVBYAnhVmw2XakgBRJCQruViEoliZURXHFGseBCBYIKZjFOMZZSYCWEUkJRq5Kb8XPpvjdfUSYwRpQRyZTi2FKItOmBhKLgoKRw6RDEwUjBCAjkL9nQFu6l/WKKncbdgwZdzuK33p1kZb1vw0t6tSR3X3Rd83fzXvToOk86SxKLkXXQcDOrhbijsY6y2LAKEseFHHf8Kp1xuOfyKUGLFVGpk8SUq2%2BrnUyfo8rlcpxH86wy7zxHeWKqN9Jlv1OZ/YQ6RfEaWC7/AkvRHGW%2BBwAA"
dash = pn.Column(
    # spinner,
    # pn.pane.Markdown(f"## Core field:\n[(View in VirES)]({vires_core_url})"),
    dmap_core,
    # pn.pane.Markdown(f"## Crustal field:\n[(View in VirES)]({vires_crust_url})"),
    dmap_crust,
    dmap_mio,
    dmap_mma,
)

In [ ]:
# bug when running? run this twice to fix it
dash.servable(title="Geomagnetic Model Explorer")